In [146]:
import os
import re
import pickle
import random
import pathlib
from typing import List, Tuple, Any

from dotenv import load_dotenv
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory


load_dotenv("../.envrc")

True

- Bot定義
- キャラクター生成
  - BotがLINEにフレンド追加されたとき
  - キャラクター記憶、キャラクター名を保存
- 世界観送る
- 挨拶メッセージ送る

-- 会話開始 --

- メッセージ受信
  - user_id, display_name取得
- キャラクター記憶呼び出し
  - 会話記憶呼び出し
- 次の会話を生成
- 返答(LINE API)

In [161]:
class ChatBot:
    def __init__(self, partner_user_id: str, partner_user_name: str) -> None:
        self.partner_user_id = partner_user_id
        self.partner_user_name = partner_user_name
        self.gen_charactor_filepath = "../prompts/generate_charactor.txt"
        self.gen_first_message_filepath = "../prompts/start_conversation.txt"

        self.llm = ChatOpenAI(
            model_name="gpt-3.5-turbo",
            top_p=0.5,
            temperature=0.8,
            frequency_penalty=2,
            max_tokens=2000,
        )  # type: ignore

        self.charactor_name = ""
        self.save_dir = pathlib.Path(f"../data/{partner_user_id}")
        if self.save_dir.exists():
            self.charactor_name = self._load_txt(str(self.save_dir / "charactor_name"))
            self.memory = self.load_pickle(str(self.save_dir / "memory.pkl"))
            self.conversation  = self._get_conversation(self.memory)
        else:
            self.save_dir.mkdir(exist_ok=True)

            while len(self.charactor_name) == 0:
                self.backgroud = self.gen_background()
                self.charactor_name = self._get_charactor_name(self.backgroud)

            self._save_txt(str(self.save_dir / "background"), self.backgroud)
            self._save_txt(str(self.save_dir / "charactor_name"), self.charactor_name)

            self.memory = ConversationBufferWindowMemory(k=20, return_messages=True)
            self.conversation  = self._get_conversation(self.memory)

            self.save_pickle(str(self.save_dir / "memory.pkl"), self.memory)

    def load_pickle(self, filepath: str) -> Any:
        with open(filepath, "rb") as f:
            data = pickle.load(f)
        return data

    def save_pickle(self, filepath: str, data: Any) -> None:
        with open(filepath, "wb") as f:
            pickle.dump(data, f)

    def _load_txt(self, filepath) -> str:
        with open(filepath, "r") as file:
            txt = file.read()
        return txt

    def _save_txt(self, filepath, txt) -> str:
        with open(filepath, "w") as file:
            file.write(txt)
    
    def gen_background(self) -> str:
        world = self._get_age()
        prompt = self._load_txt(self.gen_charactor_filepath)
        prompt = prompt.format(age=world)

        messages = [SystemMessage(content=prompt), HumanMessage(content="Start")]
        res = self.llm(messages)
        content  = res.content
        return content

    def _get_age(self) -> List[str]:
        age = random.choices(
            ["現代", "ファンタジー世界", "ダークファンタジー", "SF", "時代劇", "中世", "産業革命期", "スチームパンク",]
        )
        return age
    
    def _get_charactor_name(self, greeting_message: str) -> str:
        match = re.search(
            r"\[ヒロイン名\]=: (.+)",
            greeting_message,
        )
        charactor_name = ""
        if match:
            charactor_name = match.group(1)
            charactor_name = re.sub(r"\s", "", charactor_name)  # スペースも除去する
        return charactor_name

    def _get_conversation(
            self, memory: ConversationBufferWindowMemory
        ) -> ConversationChain:
        system_prompt = self._load_txt(self.gen_first_message_filepath)
        system_prompt = system_prompt.format(
            charactor_name=self.charactor_name,
            partner_user_name=self.partner_user_name,
        )
        prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessagePromptTemplate.from_template(system_prompt),
                MessagesPlaceholder(variable_name="history"),
                HumanMessagePromptTemplate.from_template("{input}"),
            ]
        )

        conversation = ConversationChain(
            verbose=False, memory=memory, prompt=prompt, llm=self.llm
        )  # type: ignore
        return conversation
    
    def talk(self, message: str) -> str:
        response = self.conversation.run(input=message)
        return response




In [169]:
bot = ChatBot("12sdflk3sd5lksdu", "konumaru")

In [163]:
first_message = bot.talk("")

In [164]:
print(first_message)

美咲: 私は大学生で、音楽が好きです。最近はピアノを習い始めたんですよ。
konumaru: そうなんですか！私も昔ピアノを習っていました。美咲さんのお気に入りの曲はありますか？
美咲:
選択肢:
1. 「Moon River」
2. 「Yesterday」
3. 「A Thousand Miles」
4. 「Let It Be」
5. 自由に行動を入力してもよい


In [171]:
message = bot.talk("1")
print(message)

美咲: そうですね、私がおすすめするのはショパンの「革命」エチュードです。力強くて情熱的な曲で、聴いていると胸が高鳴りますよ。
konumaru: 革命エチュード…興味津々ですね。今度聞いてみます！
美咲:
選択肢:
1. 私もその曲大好きだから一緒に聴きましょう！
2. もし良かったらあなたもピアノを弾けるように教えませんか？
3. 音楽って感情表現する上で本当に大切だと思います。
4. 最近流行ってる歌手やバンド知ってます？私も新しい音楽探したいんだけど…
5. 自由回答
